In [47]:
import pandas as pd
import numpy as np
import time
from MyNLP import WordDividerMecab
from sklearn.feature_extraction.text import TfidfVectorizer

In [48]:
df = pd.read_csv("data/2022_12_20/news_list_12_20_business_topic_500.csv", encoding="cp932", header=None)
comment_list = []
row_data = df[2][1:].to_numpy().tolist()

temp_row_data = []
for text in row_data:
    if len(text) >= 0:
        temp_row_data.append(text)
row_data = temp_row_data

# 前処理
wd = WordDividerMecab()
stop_word_list = ["まだ", "ある", "なる", "なる", "する", "し", "する", "いる", "なっ", "せ", "い", "やる", "ない", "画像", "参照"]
for text in row_data:
    text = wd.wakati_text_delete(text=text, stop_word_list=stop_word_list)
    comment_list.append(text)

# 単語の集合(vocablury)の作成
vectorizer = TfidfVectorizer()
tfidf = vectorizer.fit_transform(comment_list)
voc = vectorizer.vocabulary_
voc = sorted(voc)

# HLDAの入力に合わせたリストの形状に変換する
corpus = []

for comment in comment_list:
    list = comment.split(" ")
    corpus.append(list)
filtered_corpus = []
for comment in corpus:
    filtered_comment = []
    for word in comment:
        if word in voc:
            filtered_comment.append(word)
    filtered_corpus.append(filtered_comment)

# 前処理で文字数がなくなったテキストがあるかのチェック
corpus = filtered_corpus
corpus = [comment for comment in corpus if len(comment) != 0]

# 単語の集合に対してindexを割り当てる
vocab_index = {}
for i, w in enumerate(voc):
    vocab_index[w] = i

# HLDAの入力に合わせてテキストを単語のindexで表す
new_corpus = []
corpus = [comment for comment in corpus if comment != '']
for sentence in corpus:
    new_sentence = []
    for word in sentence:
        word_idx = vocab_index[word]
        new_sentence.append(word_idx)
    new_corpus.append(new_sentence)

In [43]:
from hlda.sampler import HierarchicalLDA


n_samples = 500       # no of iterations for the sampler
alpha = 10.0          # smoothing over level distributions
gamma = 10.0           # CRP smoothing parameter; number of imaginary customers at next, as yet unused table
eta = 0.1            # smoothing over topic-word distributions
num_levels = 3        # the number of levels in the tree
display_topics = 100   # the number of iterations between printing a brief summary of the topics so far
n_words = 10           # the number of most probable words to print for each topic after model estimation
with_weights = False  # whether to print the words with the weights

In [49]:
from hlda.sampler import HierarchicalLDA


n_samples = 500       # no of iterations for the sampler
alpha = 10.0          # smoothing over level distributions
gamma = 10.0           # CRP smoothing parameter; number of imaginary customers at next, as yet unused table
eta = 0.1            # smoothing over topic-word distributions
num_levels = 4        # the number of levels in the tree
display_topics = 100   # the number of iterations between printing a brief summary of the topics so far
n_words = 10           # the number of most probable words to print for each topic after model estimation
with_weights = False  # whether to print the words with the weights

In [ ]:
from hlda.sampler import HierarchicalLDA


n_samples = 500       # no of iterations for the sampler
alpha = 10.0          # smoothing over level distributions
gamma = 10.0           # CRP smoothing parameter; number of imaginary customers at next, as yet unused table
eta = 0.1            # smoothing over topic-word distributions
num_levels = 4        # the number of levels in the tree
display_topics = 100   # the number of iterations between printing a brief summary of the topics so far
n_words = 10           # the number of most probable words to print for each topic after model estimation
with_weights = False  # whether to print the words with the weights

In [50]:
hlda = HierarchicalLDA(new_corpus, voc, alpha=alpha, gamma=gamma, eta=eta, num_levels=num_levels)
hlda.estimate(n_samples, display_topics=display_topics, n_words=n_words, with_weights=with_weights)

HierarchicalLDA sampling

.................................................................................................... 100
topic=0 level=0 (documents=475): できる, いう, 発表, 日本, 19日, 見る, 必要, 企業, 2022年, 行う, 
    topic=1 level=1 (documents=52): モデル, 新型, 搭載, 採用, デザイン, 価格, 設定, 使う, 最大, 専用, 
        topic=2 level=2 (documents=8): 扶養, 所得, 住民税, 税金, 世帯, 非課税, 家族, 金額, 場合, ふるさと納税, 
            topic=234 level=3 (documents=8): 生活保護, 場合, 自動車, 年金, 持ち家, 収入, 生活, 受給, 売却, 独身者, 
        topic=503 level=2 (documents=44): トヨタ, タイ, シート, 水素, スタンド, システム, ミニバン, カーボンニュートラル, バッテリー, 自転車, 
            topic=504 level=3 (documents=15): smartphone, 製品, カメラ, クリップ, 調理, 本体, 背面, ヘッドセット, 投入, 送信機, 
            topic=607 level=3 (documents=29): エンジン, 装備, クルマ, タイヤ, モデル, フォーチュナー, 性能, スポーツ, 空気, ボディ, 
    topic=7 level=1 (documents=7): 柔軟, ガス料金, 報告, 9月, 往来, 体験, 足元, 出席者, 入力, 11月19日, 
        topic=8 level=2 (documents=7): ツアー, 北京, 引き上げる, 1回, 南側, 大学, 住民, 訪日外国人, 2週間, 新型コロナウイルス, 
            topic=9 level=3 (documents=7): 旅行, コロ

In [45]:
import pickle
import gzip

def save_zipped_pickle(obj, filename, protocol=-1):
    with gzip.open(filename, 'wb') as f:
        pickle.dump(obj, f, protocol)
        
def load_zipped_pickle(filename):
    with gzip.open(filename, 'rb') as f:
        loaded_object = pickle.load(f)
        return loaded_object

In [46]:
save_zipped_pickle(hlda, 'model/2022_12_20/yahoo_business_topic_500news_hlda_level3_gamma10.pickle')
#save_zipped_pickle(hlda, '2022_11_15/yahoo_hlda_default.pickle')

In [28]:
hlda = load_zipped_pickle('model/2022_12_20/yahoo_business_topic_1000news_hlda_level3_gamma10.pickle')
#hlda = load_zipped_pickle('2022_11_15/yahoo_hlda_default.pickle')

In [32]:
hlda.print_nodes(n_words = 12, with_weights = False)

topic=0 level=0 (documents=928): できる, いう, 発表, 日本, 見る, 企業, 必要, 行う, 2022年, 高い, 場合, 多い, 
    topic=7 level=1 (documents=269): 利上げ, インフレ, 示す, 目標, 金利, 述べる, 見通し, 機能, 経済, 2023年, 変動, 長期, 
        topic=38 level=2 (documents=145): 日銀, 長期金利, 金利, 修正, 20日, 金融緩和, 総裁, 上昇, 市場, 一時, 拡大, 変動, 
        topic=122 level=2 (documents=66): モデル, 搭載, 採用, 装備, 新型, システム, スポーツ, グレード, 可能, トヨタ, 走行, 機能, 
        topic=144 level=2 (documents=9): 増税, 防衛費, 評価, 方針, 支持層, 増額, ブラックロック, 首相, 反撃, 能力, 安保, 泉沢, 
        topic=156 level=2 (documents=20): 開催, イベント, 展示, 車両, レース, 東京オートサロン, 15日, 落札, コンテンツ, カスタム, クルマ, マツダ, 
        topic=201 level=2 (documents=22): 販売, 商品, 値上げ, 店舗, ピース, ソース, ホットミルク, 大人, 抹茶, 薄皮, 牛乳, ローソン, 
        topic=382 level=2 (documents=7): スタジアム, モビリティ, 体験, 産業, ソフトバンク, シティ, ニューヨーク市, 特許, 知的財産権, アプリ, キャンペーン, ウェイモ, 
    topic=9 level=1 (documents=57): ドコモ, 指定, 結論, 開始, 連続, 三菱, 5か月, 残高, 川越, 昭和, 新幹線, 位置づける, 
        topic=18 level=2 (documents=8): 予算案, 当初, 調整, 総額, 来年度, 過去最大, 国債, 4000億円, 政府, 税収, 一般会計, 今年度, 
        topic